Gang SHITE

In [ ]:
!pip install html2text --quiet


In [ ]:

!pip install simpletransformers --quiet

     |████████████████████████████████| 122kB 3.4MB/s 
     |████████████████████████████████| 491kB 10.6MB/s 
     |████████████████████████████████| 204kB 58.5MB/s 
     |████████████████████████████████| 3.8MB 56.6MB/s 
     |████████████████████████████████| 1.0MB 49.7MB/s 
     |████████████████████████████████| 870kB 38.7MB/s 


In [ ]:
import numpy as np
import requests
import html2text
from googlesearch import search
import re
from simpletransformers.question_answering import QuestionAnsweringModel
from IPython.display import display
from IPython.html import widgets
from bs4 import BeautifulSoup
from markdown import markdown 

/usr/local/lib/python3.6/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [ ]:
model = QuestionAnsweringModel('distilbert','distilbert-base-uncased-distilled-squad')

In [ ]:
question_data = {
  'qas': 
  [{'question': 'What color is the sky?',
    'id': 0,
    'answers': [{'text': ' ', 'answer_start': 0}],
    'is_impossible': False}],
  'context': 'the sky is blue'
  }

prediction = model.predict([question_data])
print(prediction)

100%|██████████| 1/1 [00:00<00:00, 755.59it/s]

Converting to features started.



[{'id': 0, 'answer': 'blue'}]


In [ ]:
def predict_answer(model, question, contexts, seq_len=512, debug=False):
    split_context = []
    
    if not isinstance(contexts, list):
        contexts = [contexts]
    
    for context in contexts:
        for i in range(0, len(context), seq_len):
            split_context.append(context[i:i+seq_len])
            
    split_context = contexts
    
    f_data = []
    
    for i, c in enumerate(split_context):
        f_data.append(
            {'qas': 
              [{'question': question,
               'id': i,
               'answers': [{'text': ' ', 'answer_start': 0}],
               'is_impossible': False}],
              'context': c
            })
        
    prediction = model.predict(f_data)
    if debug:
        print(prediction)
    preds = [x['answer'].lower().strip() for x in prediction if x['answer'].strip() != '']
    if preds:
      return max(set(preds), key=preds.count)
    return 'No answer'

In [ ]:
predict_answer(model,'what is siddharth ?',[''])

100%|██████████| 1/1 [00:00<00:00, 402.87it/s]

Converting to features started.


'cool'

In [ ]:
# Source: https://gist.github.com/lorey/eb15a7f3338f959a78cc3661fbc255fe
def markdown_to_text(markdown_string):
    """ Converts a markdown string to plaintext """

    # md -> html -> text since BeautifulSoup can extract text cleanly
    html = markdown(markdown_string)

    # remove code snippets
    html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
    html = re.sub(r'<code>(.*?)</code >', ' ', html)

    # extract text
    soup = BeautifulSoup(html, "html.parser")
    text = ''.join(soup.findAll(text=True))

    return text

def format_text(text):
    text = markdown_to_text(text)
    text = text.replace('\n', ' ')
    return text

In [ ]:
links = list(search('what color is the sky?', stop=2))
print(links)

html_conv = html2text.HTML2Text()
html_conv.ignore_links = True
html_conv.escape_all = True

text = []
for link in links:
    req = requests.get(link)
    text.append(html_conv.handle(req.text))
    text[-1] = format_text(text[-1])

print(text)

In [ ]:
def query_pages(query, n=5):
    return list(search(query, stop=n))

def query_to_text(query, n=5):
    html_conv = html2text.HTML2Text()
    html_conv.ignore_links = True
    html_conv.escape_all = True
    
    text = []
    for link in query_pages(query, n):
        req = requests.get(link)
        text.append(html_conv.handle(req.text))
        text[-1] = format_text(text[-1])
        
    return text

In [ ]:
question = 'What is the color of an apple?'
context = query_to_text(question, n=2)
pred = predict_answer(model, question, context)
print(pred)

100%|██████████| 2/2 [00:00<00:00, 41.39it/s]

Converting to features started.



red


In [ ]:
def q_to_a(model, question, n=2, debug=False):
    context = query_to_text(question, n=n)
    pred = predict_answer(model, question, context, debug=debug)
    return pred

In [ ]:
q_to_a(model,'Who is obama ?')

  0%|          | 0/2 [00:00<?, ?it/s]

Converting to features started.


100%|██████████| 2/2 [00:06<00:00,  3.05s/it]


'the only president born outside the contiguous 48 states'

In [ ]:
 text = widgets.Text(description='Question:', width=300)
display(text)

button = widgets.Button(description='Get an Answer')
display(button)

def on_button_click(b):
    answer = q_to_a(model, text.value, n=2)
    print('Answer:', answer)
    
button.on_click(on_button_click)

Text(value='', description='Question:')

Button(description='Get an Answer', style=ButtonStyle())

  0%|          | 0/2 [00:00<?, ?it/s]

Converting to features started.


100%|██████████| 2/2 [00:02<00:00,  1.84s/it]



Answer: incumbent prime minister of india is narendra modi
